# 2장. 단일 책임 원칙을 따르는 클래스 디자인하기

객체지향 시스템은 클래스들 사이에 주고 받는 메세지로 이루어진다

클래스를 만들기 위해서는(20 p)
1. 연관성 있는 메서드들을 포함하도록
2. 수정하기 쉽도록

수정하기 쉬운 코드의 특징(21 p)
1. 투명하다
2. 적절하다
3. 사용가능하다
4. 모범이 된다

### 하나의 책임만 가지는 클래스를 만들어보자

### Page 22

In [43]:
chainring = 52  # 앞 톱니 개수
cog = 11  # 뒷 톱니 개수
ratio = chainring / float(cog)  # 기어비
print(ratio)

chainring = 30
cog = 27
ratio = chainring / float(cog)
print(ratio)

4.7272727272727275
1.1111111111111112


놀랍도록 기어비에 신경쓰는 사람들이 있습니다. 그게 저는 아닙니다만 
http://www.bikem.co.kr/article/read.php?num=8519

### Page 23

In [53]:
class Gear(object):
    def __init__(self, chainring, cog):
        """
        앞, 뒤 톱니바퀴수
        """
        self.__chainring = chainring
        self.__cog = cog

    def ratio(self):
        return self.__chainring / float(self.__cog)

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

In [54]:
def test_gear_ratio():
    print(Gear(52, 11).ratio())
    print(Gear(30, 27).ratio())

In [55]:
test_gear_ratio()

4.7272727272727275
1.1111111111111112


아무 문제 없어 보이지만, 바퀴의 크기가 다른 경우가 있다. 예를 들면 이런 경우?  
26인치 vs 29인치 : http://m.blog.naver.com/searider/220055184841  
폴딩 : http://blog.naver.com/PostView.nhn?blogId=ninjarazzi&logNo=40117828380

새로운 행동을 추가하자

### Page 20

In [57]:
class Gear(object):
    def __init__(self, chainring, cog, rim, tire):
        """
        앞, 뒤 톱니바퀴수
        바퀴테 지름
        타이어 높이
        """
        self.__chainring = chainring
        self.__cog = cog
        self.__rim = rim
        self.__tire = tire

    def ratio(self):
        return self.chainring / float(self.cog)

    def gear_inches(self):
        # 타이어는 바퀴를 감싸고 있으므로, 지름을 계산할 때 타이어 높이에 2를 곱한다
        return self.ratio() * (self.rim + (self.tire * 2))

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

    @property
    def rim(self):
        return self.__rim

    @property
    def tire(self):
        return self.__tire

In [58]:
print(Gear(52, 11, 26, 1.5).gear_inches())
print(Gear(52, 11, 24, 1.25).gear_inches())

137.0909090909091
125.27272727272728


In [59]:
# 잘 되던게 안되는 이유
test_gear_ratio()

TypeError: __init__() missing 2 required positional arguments: 'rim' and 'tire'

클래스란 쉽게 가져다 쓸 수 있는 코드  
결합도가 높으면 유지보수가 힘들어 진다  
<br>  
행동에 기반한 코드를 만들어보자

### Page 30

In [60]:
class Gear(object):
    def __init__(self, chainring, cog):
            self.__chainring = chainring
            self.__cog = cog

    def ratio(self):
        return self.__chainring / float(self.__cog)      # 멸망의 길

In [61]:
test_gear_ratio()

4.7272727272727275
1.1111111111111112


### Page 30

In [68]:
class Gear(object):
    def __init__(self, chainring, cog):
        self.__chainring = chainring
        self.__cog = cog

    def ratio(self):
        """
        변수를 감싸고 이를 사용하자
        """
        return self.chainring / float(self.cog)

    @property
    def chainring(self):   
        """
        앞 톱니바퀴수가 무엇인지 아는 프로퍼티를 만든다. 
        계산식이 바뀌더라도 이 부분만 바꿔주면 된다(31 p)
        """
        return self.__chainring

    @property
    def cog(self):
        """
        뒷 톱니바퀴수가 무엇인지 아는 프로퍼티를 만든다. 
        """
        return self.__cog

In [69]:
test_gear_ratio()

4.7272727272727275
1.1111111111111112


### Page 32 복잡한 데이터 구조에 의존하는 것은 더 안좋다

In [70]:
class ObscuringReferences(object):
    def __init__(self, data):
        self._data = data

    def diameters(self):
        # 0 is rim, 1 is tire
        return [cell[0] + (cell[1] * 2) for cell in self.data]

    @property
    def data(self):
        return self._data

    # ... many other methods that index into the array

In [71]:
data = [[622, 20], [622, 23], [559, 30], [559, 40]]

In [73]:
ors = ObscuringReferences(data)

In [75]:
# 지금은 잘 동작하지만 배열 구조가 바뀌면 클래스가 가지고 있는 데이터 구조를 바꾸는 것이기 때문에 영향도가 크다
ors.diameters()

[662, 668, 619, 639]

### Page 28

In [32]:
from collections import namedtuple

In [33]:
class RevealingReferences(object):
    def __init__(self, data):
        self.wheels = data

    def diameters(self):
        return [wheel.rim + (wheel.tire * 2) for wheel in self.wheels]

    @property
    def wheels(self):
        return self.__wheels

    Wheel = namedtuple('Wheel', ['rim', 'tire'])

    @wheels.setter
    def wheels(self, value):
        self.__wheels = [self.Wheel(cell[0], cell[1]) for cell in value]

### Page 29

In [34]:
class RevealingReferences(object):
    def __init__(self, data):
        self.wheels = data
    
    # page 29
    # first - iterate over the array
    def diameters(self):
        return [self.diameter(wheel) for wheel in self.wheels]

    # second - calculate diameter of ONE wheel
    def diameter(self, wheel):
        return wheel.rim + (wheel.tire * 2)
    
    @property
    def wheels(self):
        return self.__wheels

    Wheel = namedtuple('Wheel', ['rim', 'tire'])

    @wheels.setter
    def wheels(self, value):
        self.__wheels = [self.Wheel(cell[0], cell[1]) for cell in value]

### Page 32

In [36]:
class Gear(object):
    def __init__(self, chainring, cog, rim, tire):
        self.__chainring = chainring
        self.__cog = cog
        self.__wheel = self.Wheel(rim, tire)

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

    @property
    def wheel(self):
        return self.__wheel

    def ratio(self):
        return self.chainring / float(self.cog)

    def gear_inches(self):
        return self.ratio() * self.wheel.diameter()

    class Wheel(namedtuple('Wheel', ['rim', 'tire'])):
        def diameter(self):
            return self.rim + (self.tire * 2)

### Page 33

In [38]:
class Gear(object):
    def __init__(self, chainring, cog, wheel=None):
        self.__chainring = chainring
        self.__cog = cog
        self.__wheel = wheel

    @property
    def chainring(self):
        return self.__chainring

    @property
    def cog(self):
        return self.__cog

    @property
    def wheel(self):
        return self.__wheel

    def ratio(self):
        return self.chainring / float(self.cog)

    def gear_inches(self):
        return self.ratio() * self.wheel.diameter()


class Wheel(object):
    def __init__(self, rim, tire):
        self.__rim = rim
        self.__tire = tire

    def diameter(self):
        return self.rim + (self.tire * 2)

    def circumference(self):
        import math
        return self.diameter() * math.pi

    @property
    def rim(self):
        return self.__rim

    @property
    def tire(self):
        return self.__tire

In [39]:
wheel = Wheel(26, 1.5)

print(wheel.circumference())
print(Gear(52, 11, wheel).gear_inches())
print(Gear(52, 11).ratio())

91.106186954104
137.0909090909091
4.7272727272727275
